In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import numpy as np

In [21]:
!ls /content/drive/MyDrive/ego4d_data_annot/annotations

fho_hands_test_unannotated.json  fho_hands_val.json  manifest.ver
fho_hands_train.json		 manifest.csv


In [22]:
import json
 
# Opening JSON file
with open('/content/drive/MyDrive/ego4d_data_annot/annotations/fho_hands_val.json') as json_file:
    gt_data = json.load(json_file)
 
# Print the type of data variable
print("Type:", type(gt_data))

Type: <class 'dict'>


In [23]:
import json
 
# Opening JSON file
with open('/content/drive/MyDrive/submission_validation_num_8.json') as json_file:
    pred_data = json.load(json_file)
 
# Print the type of data variable
print("Type:", type(pred_data))

Type: <class 'dict'>


In [24]:
# clip_uid = '0bc8996b-85d5-4800-b840-f7ef647a21ee'
# clip_uid = '1b2f1101-b3d9-4be5-bc92-393c983be9b4'
clip_uid = '3def2afc-55d2-458e-b792-d15cd291645b'

In [25]:
gt_results = {}
gt_labels = {}
gt_label_masks = {}
frame_clips = {}
for clip in gt_data['clips']:
  if clip['clip_uid'] == clip_uid:
  # if True:
    clip_id = clip['clip_id']
    for annot in clip['frames']:
      pre45_frame = annot['pre_45']['frame']
      clip_name = str(clip_id) + '_' + str(pre45_frame - 1)      

      label = [0.0] * 20
      label_mask = [0.0] * 20
      frame_nums = [None] * 5
      for frame_type, frame_annot in annot.items():
          if frame_type in ["action_start_sec", "action_end_sec", "action_start_frame",
                            "action_end_frame", "action_clip_start_sec", "action_clip_end_sec",
                            "action_clip_start_frame", "action_clip_end_frame"]:
              continue
          if len(frame_annot) == 2:
              continue
          frame_gt = frame_annot['boxes']
          if frame_type == 'pre_45':
              frame_nums[0] = annot['pre_45']['clip_frame']
              for single_hand in frame_gt:
                  if 'left_hand' in single_hand:
                      label_mask[0] = 1.0
                      label_mask[1] = 1.0
                      label[0] = single_hand['left_hand'][0]
                      label[1] = single_hand['left_hand'][1]
                  if 'right_hand' in single_hand:
                      label_mask[2] = 1.0
                      label_mask[3] = 1.0
                      label[2] = single_hand['right_hand'][0]
                      label[3] = single_hand['right_hand'][1]
          if frame_type == 'pre_30':
              frame_nums[1] = annot['pre_30']['clip_frame']
              for single_hand in frame_gt:
                  if 'left_hand' in single_hand:
                      label_mask[4] = 1.0
                      label_mask[5] = 1.0
                      label[4] = single_hand['left_hand'][0]
                      label[5] = single_hand['left_hand'][1]
                  if 'right_hand' in single_hand:
                      label_mask[6] = 1.0
                      label_mask[7] = 1.0
                      label[6] = single_hand['right_hand'][0]
                      label[7] = single_hand['right_hand'][1]
          if frame_type == 'pre_15':
              frame_nums[2] = annot['pre_15']['clip_frame']
              for single_hand in frame_gt:
                  if 'left_hand' in single_hand:
                      label_mask[8] = 1.0
                      label_mask[9] = 1.0
                      label[8] = single_hand['left_hand'][0]
                      label[9] = single_hand['left_hand'][1]
                  if 'right_hand' in single_hand:
                      label_mask[10] = 1.0
                      label_mask[11] = 1.0
                      label[10] = single_hand['right_hand'][0]
                      label[11] = single_hand['right_hand'][1]
          if frame_type == 'pre_frame':
              frame_nums[3] = annot['pre_frame']['clip_frame']
              for single_hand in frame_gt:
                  if 'left_hand' in single_hand:
                      label_mask[12] = 1.0
                      label_mask[13] = 1.0
                      label[12] = single_hand['left_hand'][0]
                      label[13] = single_hand['left_hand'][1]
                  if 'right_hand' in single_hand:
                      label_mask[14] = 1.0
                      label_mask[15] = 1.0
                      label[14] = single_hand['right_hand'][0]
                      label[15] = single_hand['right_hand'][1]
          if frame_type == 'contact_frame':
              frame_nums[4] = annot['contact_frame']['clip_frame']
              for single_hand in frame_gt:
                  if 'left_hand' in single_hand:
                      label_mask[16] = 1.0
                      label_mask[17] = 1.0
                      label[16] = single_hand['left_hand'][0]
                      label[17] = single_hand['left_hand'][1]
                  if 'right_hand' in single_hand:
                      label_mask[18] = 1.0
                      label_mask[19] = 1.0
                      label[18] = single_hand['right_hand'][0]
                      label[19] = single_hand['right_hand'][1]
      annot = list(np.multiply(label, label_mask))
      gt_results[clip_name] = annot
      gt_labels[clip_name] = label
      gt_label_masks[clip_name] = label_mask
      frame_clips[clip_name] = frame_nums


In [17]:
# video_id = '3004_9349'
# clip_id = '3004_16165'

# clip_id = '3353_5186'

clip_id = '611_24025'

In [18]:
import cv2
import os
cap = cv2.VideoCapture(f'/content/drive/MyDrive/{clip_uid}.mp4')

mask = gt_label_masks[clip_id]
pred = pred_data[clip_id]
label = gt_labels[clip_id]

pre45_frame = frame_clips[clip_id]
for key_frame in range(5):
    frame = frame_clips[clip_id][key_frame]
    if frame == None:
      continue
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1)
    res, img = cap.read()
    new_height = 256
    new_width = int(img.shape[1] * (new_height / img.shape[0]))
    dim = (new_width, new_height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    for single_hand in range(2):
        if mask[key_frame * 4 + single_hand * 2]:
            cd_pred = (
                int(pred[key_frame * 4 + single_hand * 2]),
                int(pred[key_frame * 4 + single_hand * 2 + 1]),
            )
            cd_label = (
                int(label[key_frame * 4 + single_hand * 2]),
                int(label[key_frame * 4 + single_hand * 2 + 1]),
            )
            cv2.circle(img, cd_pred, 5, (255, 0, 0), thickness=-1)
            cv2.circle(img, cd_label, 5, (0, 255, 0), thickness=-1)
    output_path = str(frame).zfill(6)
    folder_name = f'./{clip_id}'
    if not os.path.exists(folder_name):
      os.makedirs(folder_name)
    cv2.imwrite(f'{folder_name}/{output_path}.png', img)
